In [3]:
import pandas as pd
import numpy as np
import ast 
import json
import requests
from datetime import datetime
from datetime import date
import os
import sys

In [49]:
# Set your API key here
API_KEY = 'Vz7nSide5OocT948BbImlJTkCKREvs6mrTBf2AL5'
# S3 bucket url base + s3Path (in get request) = the full path to the files
####### Streaming services API #######
# This is an example of how to use the streaming services API to get hourly emissions data.

# Use this API for continuous data streams and avoiding paging through results.

# api parameters for the streaming emissions/apportioned/hourly endpoint

for k in range(0,22):
    main_df = []
    if k<10:
        k_str = '0'+str(k)
    else:
        k_str = str(k)
    for j in range(1,13):
        for i in range(1,32):
            if (i<10) & (j<10):
                parameters = {
                    'api_key': API_KEY,
                    'beginDate': '20'+k_str+'-0'+str(j)+'-0'+str(i),
                    'endDate': '20'+k_str+'-0'+str(j)+'-0'+str(i),
                    'programCodeInfo': 'CSOSG2|CSOSG3',
                }
            elif (i<10) & (j>=10):
                parameters = {
                    'api_key': API_KEY,
                    'beginDate': '20'+k_str+'-'+str(j)+'-0'+str(i),
                    'endDate': '20'+k_str+'-'+str(j)+'-0'+str(i),
                    'programCodeInfo': 'CSOSG2|CSOSG3',
                }
            elif (i>=10) & (j<10):
                parameters = {'api_key': API_KEY,
                    'beginDate': '20'+k_str+'-0'+str(j)+'-'+str(i),
                    'endDate': '20'+k_str+'-0'+str(j)+'-'+str(i),
                    'programCodeInfo': 'CSOSG2|CSOSG3',
                }
            else:
                parameters = {'api_key': API_KEY,
                    'beginDate': '20'+k_str+'-'+str(j)+'-'+str(i),
                    'endDate': '20'+k_str+'-'+str(j)+'-'+str(i),
                    'programCodeInfo': 'CSOSG2|CSOSG3',
                }
    
            if (j==2&i>=29)|(j==4&i>=31)|(j==6&i>=31)|(j==9&i>=31)|(j==11&i>=31):
                continue
            
            
            date_format = "%Y-%m-%d"
            beginDate = datetime.strptime(parameters['beginDate'], date_format)
            endDate = datetime.strptime(parameters['endDate'], date_format)
            if (endDate - beginDate).days > 31:
                warnings.warn("The request you're making could be too large for the streaming services API and may result in a bad request. Please consider using the bulk data api endpoint instead. An example can be found in the bulk_data_demo.py file.")
            
            # making get request using the emissions/apportioned/hourly endpoint
            streamingUrl = "https://api.epa.gov/easey/streaming-services/emissions/apportioned/hourly"
            streamingResponse = requests.get(streamingUrl, params=parameters)
            
            # printing the response error message if the response is not successful
            print("Status code: "+str(streamingResponse.status_code))
            if (int(streamingResponse.status_code) > 399):
                sys.exit("Error message: "+streamingResponse.json()['error']['message'])
            
            #print("Field Mappings: "+str(streamingResponse.headers['X-Field-Mappings']))
            ## the upper line is for printing the list of columns
            # collecting data as a data frame
            streamingResponse_df = pd.DataFrame(streamingResponse.json())
            df_filtered = streamingResponse_df.loc[streamingResponse_df["opTime"] == 1]
            df_filtered = df_filtered.drop(["associatedStacks", "so2Mass","so2MassMeasureFlg","so2Rate","so2RateMeasureFlg",\
                                           "co2Mass","co2MassMeasureFlg","co2Rate","co2RateMeasureFlg","noxMass",\
                                           "noxMassMeasureFlg","noxRate","noxRateMeasureFlg","heatInputMeasureFlg",\
                                           "so2ControlInfo","noxControlInfo","pmControlInfo","hgControlInfo","programCodeInfo"], axis=1)
            main_df.append(df_filtered)
    df_result = pd.concat(main_df, ignore_index=True)
    ### in case wanted to say the data in excel format, it might get to size of 10-11 meg for each year
    ### I don't think all of it can be saved in one excel file
    save_path = "your/preferred/address/year"+str(k)+".xlsx"
    df_result.to_excel(save_path, index=False, engine="openpyxl")

Status code: 200
Field Mappings: [{"label":"State","value":"stateCode"},{"label":"Facility Name","value":"facilityName"},{"label":"Facility ID","value":"facilityId"},{"label":"Unit ID","value":"unitId"},{"label":"Associated Stacks","value":"associatedStacks"},{"label":"Date","value":"date"},{"label":"Hour","value":"hour"},{"label":"Operating Time","value":"opTime"},{"label":"Gross Load (MW)","value":"grossLoad"},{"label":"Steam Load (1000 lb/hr)","value":"steamLoad"},{"label":"SO2 Mass (lbs)","value":"so2Mass"},{"label":"SO2 Mass Measure Indicator","value":"so2MassMeasureFlg"},{"label":"SO2 Rate (lbs/mmBtu)","value":"so2Rate"},{"label":"SO2 Rate Measure Indicator","value":"so2RateMeasureFlg"},{"label":"CO2 Mass (short tons)","value":"co2Mass"},{"label":"CO2 Mass Measure Indicator","value":"co2MassMeasureFlg"},{"label":"CO2 Rate (short tons/mmBtu)","value":"co2Rate"},{"label":"CO2 Rate Measure Indicator","value":"co2RateMeasureFlg"},{"label":"NOx Mass (lbs)","value":"noxMass"},{"label":

## what do next?
I think one thing that would be good to do ist to add another column of capacity of each generator, and then we would only keep observations with $\frac{gen}{cap}$ between $85-95\%$, or we  can also have efficiency dispersion histogram the same as Omer's

True